# Date: 04/13/2025

In [2]:
# Pasting these here for easier access

import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolacors
import pickle
from tqdm import tqdm  # For progress bar
from multiprocessing import Pool, cpu_count
import os
import sys
import time
import datetime 

<br>

## Objective 1: Reverted changes made to main.cpp, pkpd_artemether.cpp and pkpd_lum.cpp to 09092024
## Objective 2: Testing if --art and --AL modules give 69.5% / 96.5% efficacy for weight = 54 kg after revision

### Objective 2.1: Testing efficacy of artemether monotherapy with pmax_lum = 0

In [19]:
# Function to run a single simulation for a given weight value

def run_single_weight_simulation(weight):
    start_time = time.monotonic()
    command_weight = [
        "/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk",
        "--AL",
        "-n", "10000",
        "--pmax_lum", "0",
        "-o", "1",
        "--weight", str(weight)
    ]

    try:
        # Run the command and capture output
        output_weight = subprocess.run(command_weight, capture_output=True, text=True, check=True)

        # Process the output
        lines_weight = output_weight.stdout.splitlines()
        df_weight = pd.DataFrame([line.split(',') for line in lines_weight], columns=["PID", "HOUR", "COMP2CONC_ARTEMETHER", "COMP2CONC_LUM", "PARASITEDENSITY"])
        
        # Clean the DataFrame
        df_weight = df_weight.iloc[1:].apply(pd.to_numeric, errors='coerce')

        # Calculate the efficacy
        if df_weight['PARASITEDENSITY'].isna().any():
            failed_treatment_count = np.nan
        else:
            failed_treatment_count = df_weight[(df_weight['PARASITEDENSITY'] >= 10) & (df_weight['HOUR'] == 671.0)].shape[0]
        
        total_patients = df_weight['PID'].nunique()
        
        if total_patients == 0 or np.isnan(failed_treatment_count): 
            efficacy = np.nan
        else:
            percentage_failed = (failed_treatment_count / total_patients) * 100
            efficacy = (100 - percentage_failed)
            if np.isnan(efficacy):
                efficacy = np.nan
        
        end_time = time.monotonic()
        today = datetime.date.today()
        print("\nToday's date: ", today, "\n")
        print("Length of simulation: ", datetime.timedelta(seconds=end_time - start_time), "\n")

        return efficacy, df_weight, failed_treatment_count
        
    except subprocess.CalledProcessError as e:
        print(f"Error in subprocess for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure
    except Exception as e:
        print(f"An unexpected error occurred for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure

In [18]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [54]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-13 

Length of simulation:  0:04:28.233104 


The failed treatment count for weight 54kg is: 3171

The percentage efficacy for weight 54kg is: 68.28999999999999 



### Objective 2.2: Testing efficacy of artemether monotherapy with --art flag (to-do later)
### Objective 2.3: Testing AL efficacy with --AL flag

In [21]:
# Function to run a single simulation for a given weight value

def run_single_weight_simulation(weight):
    start_time = time.monotonic()
    command_weight = [
        "/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk",
        "--AL",
        "-n", "10000",
        "-o", "1",
        "--weight", str(weight)
    ]

    try:
        # Run the command and capture output
        output_weight = subprocess.run(command_weight, capture_output=True, text=True, check=True)

        # Process the output
        lines_weight = output_weight.stdout.splitlines()
        df_weight = pd.DataFrame([line.split(',') for line in lines_weight], columns=["PID", "HOUR", "COMP2CONC_ARTEMETHER", "COMP2CONC_LUM", "PARASITEDENSITY"])
        
        # Clean the DataFrame
        df_weight = df_weight.iloc[1:].apply(pd.to_numeric, errors='coerce')

        # Calculate the efficacy
        if df_weight['PARASITEDENSITY'].isna().any():
            failed_treatment_count = np.nan
        else:
            failed_treatment_count = df_weight[(df_weight['PARASITEDENSITY'] >= 10) & (df_weight['HOUR'] == 671.0)].shape[0]
        
        total_patients = df_weight['PID'].nunique()
        
        if total_patients == 0 or np.isnan(failed_treatment_count): 
            efficacy = np.nan
        else:
            percentage_failed = (failed_treatment_count / total_patients) * 100
            efficacy = (100 - percentage_failed)
            if np.isnan(efficacy):
                efficacy = np.nan
        
        end_time = time.monotonic()
        today = datetime.date.today()
        print("\nToday's date: ", today, "\n")
        print("Length of simulation: ", datetime.timedelta(seconds=end_time - start_time), "\n")

        return efficacy, df_weight, failed_treatment_count
        
    except subprocess.CalledProcessError as e:
        print(f"Error in subprocess for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure
    except Exception as e:
        print(f"An unexpected error occurred for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure

In [16]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [54]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-13 

Length of simulation:  0:04:37.493341 


The failed treatment count for weight 54kg is: 249

The percentage efficacy for weight 54kg is: 97.51 



<br>

### Objective 3: Debugging the dosage schedule of pkpd_artemether.cpp and observing --art and --AL efficacy for weight = 54 kg

#### Objective 3.1: Testing efficacy of artemether monotherapy with --AL and --pmax_lum = 0

In [20]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [54]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg_objective_3.1.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg_objective_3.1.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-13 

Length of simulation:  0:03:48.042578 


The failed treatment count for weight 54kg is: 0

The percentage efficacy for weight 54kg is: 100.0 



#### Objective 3.2: Testing efficacy of AL

In [22]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [54]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg_objective_3.2.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg_objective_3.2.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-13 

Length of simulation:  0:03:54.101743 


The failed treatment count for weight 54kg is: 0

The percentage efficacy for weight 54kg is: 100.0 



In [23]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [12]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg_objective_3.2.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg_objective_3.2.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-13 

Length of simulation:  0:03:47.829280 


The failed treatment count for weight 12kg is: 0

The percentage efficacy for weight 12kg is: 100.0 



<br>

### Objective 4: Testing AL efficacy after making the following changes
- `pkpd_artemether.cpp` :
    - Changed constructor to `pkpd_artemether::pkpd_artemether(patient age, patient weight)`
    - Moving log10 parasitaemia calculation from constructor to `initialize_params()`
    - Adding `set_age_and_weight()`
    - Creating `initialize()`
    - Removed `generate_recommended_dosing_schedule()` from constructor
- `pkpd_lum.cpp` :
    - Changed constructor to `pkpd_lum::pkpd_lum(patient age, patient weight)`
    - Changed `median_weight` from 54.0 to 42.0
    - Adding `set_age_and_weight()`
- `main.cpp` :
    - Added `pkpd_artemether::stochastic = true`
    - Edited the constructor call for `therapy_lumefantrine`  and `therapy_AL`

In [26]:
# Function to run a single simulation for a given weight value

def run_single_weight_simulation(weight):
    start_time = time.monotonic()
    command_weight = [
        "/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk",
        "--AL",
        "-n", "10000",
        "-o", "1",
        "--weight", str(weight)
    ]

    try:
        # Run the command and capture output
        output_weight = subprocess.run(command_weight, capture_output=True, text=True, check=True)

        # Process the output
        lines_weight = output_weight.stdout.splitlines()
        df_weight = pd.DataFrame([line.split(',') for line in lines_weight], columns=["PID", "HOUR", "COMP2CONC_ARTEMETHER", "COMP2CONC_LUM", "PARASITEDENSITY"])
        
        # Clean the DataFrame
        df_weight = df_weight.iloc[1:].apply(pd.to_numeric, errors='coerce')

        # Calculate the efficacy
        if df_weight['PARASITEDENSITY'].isna().any():
            failed_treatment_count = np.nan
        else:
            failed_treatment_count = df_weight[(df_weight['PARASITEDENSITY'] >= 10) & (df_weight['HOUR'] == 671.0)].shape[0]
        
        total_patients = df_weight['PID'].nunique()
        
        if total_patients == 0 or np.isnan(failed_treatment_count): 
            efficacy = np.nan
        else:
            percentage_failed = (failed_treatment_count / total_patients) * 100
            efficacy = (100 - percentage_failed)
            if np.isnan(efficacy):
                efficacy = np.nan
        
        end_time = time.monotonic()
        today = datetime.date.today()
        print("\nToday's date: ", today, "\n")
        print("Length of simulation: ", datetime.timedelta(seconds=end_time - start_time), "\n")

        return efficacy, df_weight, failed_treatment_count
        
    except subprocess.CalledProcessError as e:
        print(f"Error in subprocess for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure
    except Exception as e:
        print(f"An unexpected error occurred for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure

In [25]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [54]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg_objective_4.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg_objective_4.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-13 

Length of simulation:  0:03:51.179762 


The failed treatment count for weight 54kg is: 3631

The percentage efficacy for weight 54kg is: 63.690000000000005 



# Changing median weight back to 54 in lum.cpp

In [27]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [54]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg_objective_4_test01.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg_objective_4_test_01.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-14 

Length of simulation:  0:03:51.406475 


The failed treatment count for weight 54kg is: 3709

The percentage efficacy for weight 54kg is: 62.91 



### Since changing median_weight to 42.0 did not cause the drop in efficacy, opting to leave median_weight to 54.0 in pkpd_lum.cpp

# Date: 04/14/2025

<br>

### Objective 5: Editing lumefantrine dosing schedule and checking if it affects AL efficacy

In [3]:
# Function to run a single simulation for a given weight value

def run_single_weight_simulation(weight):
    start_time = time.monotonic()
    command_weight = [
        "/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk",
        "--AL",
        "-n", "10000",
        "-o", "1",
        "--weight", str(weight)
    ]

    try:
        # Run the command and capture output
        output_weight = subprocess.run(command_weight, capture_output=True, text=True, check=True)

        # Process the output
        lines_weight = output_weight.stdout.splitlines()
        df_weight = pd.DataFrame([line.split(',') for line in lines_weight], columns=["PID", "HOUR", "COMP2CONC_ARTEMETHER", "COMP2CONC_LUM", "PARASITEDENSITY"])
        
        # Clean the DataFrame
        df_weight = df_weight.iloc[1:].apply(pd.to_numeric, errors='coerce')

        # Calculate the efficacy
        if df_weight['PARASITEDENSITY'].isna().any():
            failed_treatment_count = np.nan
        else:
            failed_treatment_count = df_weight[(df_weight['PARASITEDENSITY'] >= 10) & (df_weight['HOUR'] == 671.0)].shape[0]
        
        total_patients = df_weight['PID'].nunique()
        
        if total_patients == 0 or np.isnan(failed_treatment_count): 
            efficacy = np.nan
        else:
            percentage_failed = (failed_treatment_count / total_patients) * 100
            efficacy = (100 - percentage_failed)
            if np.isnan(efficacy):
                efficacy = np.nan
        
        end_time = time.monotonic()
        today = datetime.date.today()
        print("\nToday's date: ", today, "\n")
        print("Length of simulation: ", datetime.timedelta(seconds=end_time - start_time), "\n")

        return efficacy, df_weight, failed_treatment_count
        
    except subprocess.CalledProcessError as e:
        print(f"Error in subprocess for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure
    except Exception as e:
        print(f"An unexpected error occurred for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure

In [4]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [54]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg_objective_5.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg_objective_5.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-14 

Length of simulation:  0:03:54.339982 


The failed treatment count for weight 54kg is: 3582

The percentage efficacy for weight 54kg is: 64.18 



### Changing the lumefantrine dosage do not affect the efficacy

In [5]:
patient_blood_volume = 5.5 * 10**(6) 
print(patient_blood_volume)

5500000.0


In [6]:
weight = 54
median_weight = 48.5

scaled_patient_blood_volume = patient_blood_volume * (weight / median_weight)
print(scaled_patient_blood_volume)

6123711.340206185


In [8]:
scaled_patient_blood_volume > patient_blood_volume

True

In [17]:
weight_values = np.arange(0, 100, 5)
print(weight_values)

median_weight = 48.5

for weight in weight_values:
    scaled_patient_blood_volume = patient_blood_volume * (weight / median_weight)
    print("weight", weight, " scaled_patient_blood_volume",scaled_patient_blood_volume/10**6)
    


[ 0  5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95]
weight 0  scaled_patient_blood_volume 0.0
weight 5  scaled_patient_blood_volume 0.5670103092783504
weight 10  scaled_patient_blood_volume 1.1340206185567008
weight 15  scaled_patient_blood_volume 1.7010309278350517
weight 20  scaled_patient_blood_volume 2.2680412371134016
weight 25  scaled_patient_blood_volume 2.8350515463917523
weight 30  scaled_patient_blood_volume 3.4020618556701034
weight 35  scaled_patient_blood_volume 3.9690721649484537
weight 40  scaled_patient_blood_volume 4.536082474226803
weight 45  scaled_patient_blood_volume 5.103092783505155
weight 50  scaled_patient_blood_volume 5.6701030927835046
weight 55  scaled_patient_blood_volume 6.237113402061856
weight 60  scaled_patient_blood_volume 6.804123711340207
weight 65  scaled_patient_blood_volume 7.371134020618556
weight 70  scaled_patient_blood_volume 7.938144329896907
weight 75  scaled_patient_blood_volume 8.505154639175258
weight 80  scaled_patient_blood_vo

In [20]:
weight_values = np.arange(0, 100, 5)
print(weight_values)

median_weight = 42

for weight in weight_values:
    if(weight == 0):
        new_weight = 0.1
        scaled_patient_blood_volume = patient_blood_volume * (new_weight / median_weight)
    else:
        scaled_patient_blood_volume = patient_blood_volume * (weight / median_weight)
    print("weight", weight, " scaled_patient_blood_volume",scaled_patient_blood_volume/10**6)

    


[ 0  5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95]
weight 0  scaled_patient_blood_volume 0.013095238095238097
weight 5  scaled_patient_blood_volume 0.6547619047619048
weight 10  scaled_patient_blood_volume 1.3095238095238095
weight 15  scaled_patient_blood_volume 1.9642857142857144
weight 20  scaled_patient_blood_volume 2.619047619047619
weight 25  scaled_patient_blood_volume 3.2738095238095237
weight 30  scaled_patient_blood_volume 3.928571428571429
weight 35  scaled_patient_blood_volume 4.583333333333334
weight 40  scaled_patient_blood_volume 5.238095238095238
weight 45  scaled_patient_blood_volume 5.892857142857142
weight 50  scaled_patient_blood_volume 6.5476190476190474
weight 55  scaled_patient_blood_volume 7.2023809523809526
weight 60  scaled_patient_blood_volume 7.857142857142858
weight 65  scaled_patient_blood_volume 8.511904761904763
weight 70  scaled_patient_blood_volume 9.166666666666668
weight 75  scaled_patient_blood_volume 9.821428571428573
weight 80  scaled_p

In [26]:
# Function to run a single simulation for a given weight value

def run_single_weight_simulation(weight):
    start_time = time.monotonic()
    command_weight = [
        "/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk",
        "--AL",
        "-n", "1000",
        "-o", "1",
        "--weight", str(weight)
    ]

    try:
        # Run the command and capture output
        output_weight = subprocess.run(command_weight, capture_output=True, text=True, check=True)

        # Process the output
        lines_weight = output_weight.stdout.splitlines()
        df_weight = pd.DataFrame([line.split(',') for line in lines_weight], columns=["PID", "HOUR", "COMP2CONC_ARTEMETHER", "COMP2CONC_LUM", "PARASITEDENSITY"])
        
        # Clean the DataFrame
        df_weight = df_weight.iloc[1:].apply(pd.to_numeric, errors='coerce')

        # Calculate the efficacy
        if df_weight['PARASITEDENSITY'].isna().any():
            failed_treatment_count = np.nan
        else:
            failed_treatment_count = df_weight[(df_weight['PARASITEDENSITY'] >= 10) & (df_weight['HOUR'] == 671.0)].shape[0]
        
        total_patients = df_weight['PID'].nunique()
        
        if total_patients == 0 or np.isnan(failed_treatment_count): 
            efficacy = np.nan
        else:
            percentage_failed = (failed_treatment_count / total_patients) * 100
            efficacy = (100 - percentage_failed)
            if np.isnan(efficacy):
                efficacy = np.nan
        
        end_time = time.monotonic()
        today = datetime.date.today()
        print("\nToday's date: ", today, "\n")
        print("Length of simulation: ", datetime.timedelta(seconds=end_time - start_time), "\n")

        return efficacy, df_weight, failed_treatment_count
        
    except subprocess.CalledProcessError as e:
        print(f"Error in subprocess for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure
    except Exception as e:
        print(f"An unexpected error occurred for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure

In [ ]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [54]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg_objective_6.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg_objective_6.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-14 

Length of simulation:  0:01:11.238366 


The failed treatment count for weight 54kg is: 430

The percentage efficacy for weight 54kg is: 57.0 



In [203]:
# Function to run a single simulation for a given weight value

def run_single_weight_simulation(weight):
    start_time = time.monotonic()
    command_weight = [
        "/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk",
        "--AL",
        "-n", "1000",
        "-o", "1",
        "--weight", str(weight)
    ]

    try:
        # Run the command and capture output
        output_weight = subprocess.run(command_weight, capture_output=True, text=True, check=True)

        # Process the output
        lines_weight = output_weight.stdout.splitlines()
        df_weight = pd.DataFrame([line.split(',') for line in lines_weight], columns=["PID", "HOUR", "COMP2CONC_ARTEMETHER", "COMP2CONC_LUM", "PARASITEDENSITY"])
        
        # Clean the DataFrame
        df_weight = df_weight.iloc[1:].apply(pd.to_numeric, errors='coerce')

        # Calculate the efficacy
        if df_weight['PARASITEDENSITY'].isna().any():
            failed_treatment_count = np.nan
        else:
            failed_treatment_count = df_weight[(df_weight['PARASITEDENSITY'] >= 10) & (df_weight['HOUR'] == 671.0)].shape[0]
        
        total_patients = df_weight['PID'].nunique()
        
        if total_patients == 0 or np.isnan(failed_treatment_count): 
            efficacy = np.nan
        else:
            percentage_failed = (failed_treatment_count / total_patients) * 100
            efficacy = (100 - percentage_failed)
            if np.isnan(efficacy):
                efficacy = np.nan
        
        end_time = time.monotonic()
        today = datetime.date.today()
        print("\nToday's date: ", today, "\n")
        print("Length of simulation: ", datetime.timedelta(seconds=end_time - start_time), "\n")

        return efficacy, df_weight, failed_treatment_count
        
    except subprocess.CalledProcessError as e:
        print(f"Error in subprocess for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure
    except Exception as e:
        print(f"An unexpected error occurred for weight: {weight}: {e}")
        return np.nan, None, 0  # Return NaN, None, and 0 in case of failure

In [204]:
# Define the weight values to use
#weight_values = [10, 15, 20, 30, 40]

weight_values = [54]

for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg_objective_6.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg_objective_6.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-14 

Length of simulation:  0:00:24.888768 


The failed treatment count for weight 54kg is: 0

The percentage efficacy for weight 54kg is: 100.0 



In [205]:
# Define the weight values to use
weight_values = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


for weight in weight_values:

    #Run the simulation

    efficacy, df_weight, failed_treatment_count = run_single_weight_simulation(weight)

    # Save the DataFrame to a pickle file
    df_weight.to_pickle(f"df_weight_{weight}kg_objective_6.pyobj")

    # Save the DataFrame to a CSV file
    df_weight.to_csv(f"df_weight_{weight}kg_objective_6.csv", index=False)

    print(f"\nThe failed treatment count for weight {weight}kg is:", failed_treatment_count)
    print(f"\nThe percentage efficacy for weight {weight}kg is:", efficacy, "\n")


Today's date:  2025-04-14 

Length of simulation:  0:00:22.721760 


The failed treatment count for weight 5kg is: 625

The percentage efficacy for weight 5kg is: 37.5 


Today's date:  2025-04-14 

Length of simulation:  0:00:23.005733 


The failed treatment count for weight 10kg is: 452

The percentage efficacy for weight 10kg is: 54.8 


Today's date:  2025-04-14 

Length of simulation:  0:00:25.250834 


The failed treatment count for weight 15kg is: 57

The percentage efficacy for weight 15kg is: 94.3 


Today's date:  2025-04-14 

Length of simulation:  0:00:20.792038 


The failed treatment count for weight 20kg is: 26

The percentage efficacy for weight 20kg is: 97.4 


Today's date:  2025-04-14 

Length of simulation:  0:00:24.069283 


The failed treatment count for weight 25kg is: 3

The percentage efficacy for weight 25kg is: 99.7 


Today's date:  2025-04-14 

Length of simulation:  0:00:23.896177 


The failed treatment count for weight 30kg is: 3

The percentage effica

In [207]:
weight_values = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 75]

for weight in weight_values:
    tbv = weight * (70) * (1000)
    print(f"Weight: {weight} kg, Total Blood Volume: {tbv} microliters")

Weight: 5 kg, Total Blood Volume: 350000 microliters
Weight: 10 kg, Total Blood Volume: 700000 microliters
Weight: 15 kg, Total Blood Volume: 1050000 microliters
Weight: 20 kg, Total Blood Volume: 1400000 microliters
Weight: 25 kg, Total Blood Volume: 1750000 microliters
Weight: 30 kg, Total Blood Volume: 2100000 microliters
Weight: 35 kg, Total Blood Volume: 2450000 microliters
Weight: 40 kg, Total Blood Volume: 2800000 microliters
Weight: 45 kg, Total Blood Volume: 3150000 microliters
Weight: 50 kg, Total Blood Volume: 3500000 microliters
Weight: 75 kg, Total Blood Volume: 5250000 microliters


In [38]:
mw_artemether = 48.5
THETA1_pe = 78.0
THETA2_pe = 129.0

for weight in weight_values:
    TVCL = THETA1_pe * ((weight/mw_artemether)** 0.75 )
    print("weight", weight, " TVCL",TVCL)


weight 5  TVCL 14.191091556443677
weight 10  TVCL 23.86647603670149
weight 15  TVCL 32.348693348804716
weight 20  TVCL 40.138468288001924
weight 25  TVCL 47.45077447702236
weight 30  TVCL 54.40380055030314
weight 35  TVCL 61.071604764438035
weight 40  TVCL 67.50458819431144
weight 45  TVCL 73.73907477186644
weight 50  TVCL 79.80237231748109


In [ ]:
import math

weight = 14

mw_artemether = 48.5
THETA1_pe = 78.0
THETA2_pe = 129.0
TVCL = THETA1_pe * ((weight/mw_artemether)** 0.75 )

ETA1_rv = 0.0; 
CL = TVCL * math.exp(ETA1_rv) 
CL = TVCL

TVV2 = THETA2_pe * (weight/mw_artemether) 
V2 = TVV2

ETA2_rv = np.random.normal(loc = 0, scale = math.sqrt(0.0162))
V2 *= math.exp(ETA2_rv)

    
i_artemether_k20 = CL/V2 

print("weight: ", weight)
print("CL: ", CL)
print("V2: ", V2)
print("ETA2_rv: ", ETA2_rv)
print("TVCL: ", TVCL)
print("TVV2: ", TVV2)
print("i_artemether_k20: ", i_artemether_k20)

weight:  14
CL:  30.717390974075204
V2:  46.577235227626986
ETA2_rv:  0.22380597130745325
TVCL:  30.717390974075204
TVV2:  37.23711340206185
i_artemether_k20:  0.659493652295089


In [ ]:
weight = 5
mw_artemether = 48.5
patient_blood_volume = (weight * 70 * 1000)

THETA7_pe = 0.278
THETA6_pe = -0.375

    
initial_log10_totalparasitaemia = np.log10(20000*patient_blood_volume )
TVF1 = 1.0 + THETA7_pe*(initial_log10_totalparasitaemia-3.98)
    
F1=TVF1
   
ETA4_rv = np.random.normal(loc = 0, scale = math.sqrt(0.08800))
F1 *= ETA4_rv
 
i_artemether_F1_indiv = F1
 
print("weight: ", weight)
print("patient_blood_volume: ", patient_blood_volume)
print("TVF1: ", TVF1)
print("F1: ", F1)
print("ETA4_rv: ", ETA4_rv)
print("i_artemether_F1_indiv: ", i_artemether_F1_indiv)

   
    

weight:  5
patient_blood_volume:  567010.3092783504
TVF1:  2.6887446243479705
F1:  0.9156942823495036
ETA4_rv:  0.3405657324453275
i_artemether_F1_indiv:  0.9156942823495036


In [171]:
TVMT_pe = 0.982

MT = TVMT_pe

i_artemether_KTR = 8.0/MT

ETA_rv = np.random.normal( loc = 0, scale = math.sqrt(0.23000) )
i_artemether_KTR *= math.exp(-ETA_rv);   

print("i_artemether_KTR: ", i_artemether_KTR)

i_artemether_KTR:  15.55708845125599
